## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import ast

import industryMap

## Importing data

In [2]:
# Importing File
user_df = pd.read_csv('../SavedData/Lloyds_users.csv')

# Changing the lists back to normal columns
user_df['user_biz_interaction'] = user_df['user_biz_interaction'].apply(ast.literal_eval)
user_df['user_biz_interaction_frequency'] = user_df['user_biz_interaction_frequency'].apply(ast.literal_eval)
user_df['user_biz_spend'] = user_df['user_biz_spend'].apply(ast.literal_eval)
user_df['user_user_interaction'] = user_df['user_user_interaction'].apply(ast.literal_eval)
user_df['user_user_interaction_frequency'] = user_df['user_user_interaction_frequency'].apply(ast.literal_eval)
user_df['user_user_spend'] = user_df['user_user_spend'].apply(ast.literal_eval)
user_df['user_industry_interaction'] = user_df['user_industry_interaction'].apply(ast.literal_eval)
user_df['user_industry_interaction_frequency'] = user_df['user_industry_interaction_frequency'].apply(ast.literal_eval)
user_df['user_industry_spend'] = user_df['user_industry_spend'].apply(ast.literal_eval)
user_df['user_date_transaction'] = user_df['user_date_transaction'].apply(ast.literal_eval)
user_df['user_date_transaction_freq'] = user_df['user_date_transaction_freq'].apply(ast.literal_eval)

## Get Relative Number of Users

In [3]:
user_num = len(user_df['user_id'])

# If you pick a threshold to high, some industries won't have enough unique users to build the dictionary effectively
# Anywhere above 4% starts to include anyone who has spent at an Accessory store, and in fact runs out of people to include in the list
# This causes the dictionary entries to be of different lengths
user_perc = 3
user_perc_threshold = int(round(user_num * (user_perc / 100), 0))

## Industry Big Spenders

In [4]:
industry_list = list(set(industryMap.industry_categories.values()))

industry_user_spend = {}

for industry in industry_list:
    
    user_spend_dict = {}
    top_n = user_perc_threshold
    
    for row in range(0, len(user_df)):
        if(industry in user_df['user_industry_interaction'][row]):
            industry_index = user_df['user_industry_interaction'][row].index(industry)
            industry_spend = user_df['user_industry_spend'][row][industry_index]
            
            user_spend_dict[user_df['user_id'][row]] = industry_spend
    
    sorted_user_spend_dict = dict(sorted(user_spend_dict.items(), key=lambda item: item[1], reverse=True)[:top_n])
            
    sorted_user_spend_user_list = list(sorted_user_spend_dict.keys())
    sorted_user_spend_spend_list = list(sorted_user_spend_dict.values())
            
    industry_user_spend[industry + ' users'] = sorted_user_spend_user_list
    industry_user_spend[industry + ' spends'] = sorted_user_spend_spend_list 

## Building Recommendation DataFrame

In [5]:
recommend_user_df = pd.DataFrame(data=industry_user_spend)

## Club Lloyds

In [6]:
# Getting the High Receivers and Creating Dictionary
monthly_spend_list_estimate = [x / 12 for x in user_df['received_tx_sum_gbp']]
club_lloyds_ad_dict = dict(zip(user_df['user_id'], monthly_spend_list_estimate))
sorted_club_lloyds_ad_dict = dict(sorted(club_lloyds_ad_dict.items(), key=lambda item: item[1], reverse=True)[:top_n])

# Adding this to the dataframe
recommend_user_df['Club Lloyds Target users'] = sorted_club_lloyds_ad_dict.keys()
recommend_user_df['Club Lloyds Target monthly recieves'] = sorted_club_lloyds_ad_dict.values()

In [18]:
user_df[user_df['user_id'] == 67945]['received_tx_sum_gbp']

5336    483241.3
Name: received_tx_sum_gbp, dtype: float64

## Saving Dataframe to CSV

In [21]:
recommend_user_df

,Apparel users,Apparel spends,Health and fitness users,Health and fitness spends,TV/Film users,TV/Film spends,Gaming users,Gaming spends,Children users,Children spends,...,Lunch / Light Meal users,Lunch / Light Meal spends,Accessories users,Accessories spends,Hardware users,Hardware spends,Alcohol users,Alcohol spends,Club Lloyds Target users,Club Lloyds Target monthly recieves
0,85270,5160.0,100086,240.0,8295,530.88,34568,1187.0,39596,2873.0,...,1615,1921.02,94580,757.0,63924,1154.0,33092,437.93,62396,42610.995833
1,67539,5046.0,100224,240.0,75897,517.88,95185,1125.0,19758,2808.0,...,84302,1828.73,42587,728.0,18351,1124.0,45432,433.93,67945,40270.108333
2,45079,5042.0,100658,240.0,29418,500.88,88957,1061.0,38120,2766.0,...,51233,1820.91,76607,711.0,4759,1119.0,38966,429.93,68986,39703.404167
3,45712,4993.0,100751,240.0,32858,494.88,20029,1049.0,4148,2723.0,...,37688,1820.29,90311,701.0,49137,1039.0,73744,420.92,32858,36529.708333
4,5556,4950.0,100818,240.0,40921,486.88,14894,984.0,16216,2675.0,...,97898,1803.98,56543,685.0,85948,1039.0,5876,412.91,13904,36198.741667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,82806,3944.0,37094,240.0,65935,413.88,83681,697.0,30638,2123.0,...,9303,1560.58,65884,353.0,50726,631.0,50709,240.94,59680,1683.970000
240,74438,3941.0,37177,240.0,67962,413.88,93932,697.0,51087,2123.0,...,8059,1560.29,77464,352.0,50955,629.0,60859,239.96,51295,1683.652500
241,36426,3940.0,37249,240.0,95039,413.88,73612,696.0,54200,2120.0,...,11232,1558.62,82058,352.0,70452,629.0,28056,239.95,15258,1682.393333
242,29264,3938.0,37593,240.0,31522,412.88,17741,695.0,57778,2119.0,...,20710,1558.57,94374,352.0,35646,628.0,34134,239.95,79999,1681.383333
